# Glassdoor API - Interview Difficulties 

In [1]:
import  requests
import urllib.request
import csv
import pandas as pd 
import numpy as np
import json
from bs4 import BeautifulSoup
import time

In [167]:
#Employer.csv contains all the employers retrieved via web scraping job posts. 
#Here will be using those values for the 'employer' parameter while making request via Glassdoor API
employer_df = pd.read_csv("Employer.csv")

#Reading CSV file jobTitle.csv that contains all tech topics/skills
#Here will be using those values for the 'jobTitle' parameter while making request via Glassdoor API
jobTitle_df = pd.read_csv("jobTitle.csv")

#Designation.csv contains all the designations retrieved via web scraping job posts. 
#Here will be using those values for the 'jobTitle' parameter while making request via Glassdoor API
designation_df=pd.read_csv("Designation.csv")

#designation_company.csv contains all the designations and company names retrieved via web scraping job posts. 
#Here will be using those values for the 'jobTitle' and 'employer' parameter while making request via Glassdoor API
designation_company_df=pd.read_csv('designation_company.csv')
employer_df.shape


print(employer_df.shape[0])
print(jobTitle_df.shape[0])
print(designation_df.shape[0])
print(designation_company_df.shape[0])

1579
151
4180
8490


In [77]:
jobTitle_column=[]
jobTitle_column_df=pd.DataFrame()

## Glassdoor API
We are using public Glassdoor API id (t.p=25738) and key (t.k=iRCtcWJQamE) provided via blog post: [Get Glassdoor Data from their API using Python](https://oonlab.com/get-glassdoor-data-from-their-api-using-python). This API key was gained 3 years ago when Glassdoor had free API sign up which they don’t provide any more until further notice [(link)](https://help.glassdoor.com/article/Glassdoor-API/en_US). Also, this API key has come with following restrictions on Glassdoor API Usage: [Public API Terms of Use](https://www.glassdoor.com/crs/api/glassdoor-public-api-terms.pdf). We have decided to use this public API id and key until we get our partnership inquiry permitted.
​
Note: We have submitted [partnership inquiry](https://help.glassdoor.com/ContactUs/en_US) several times during the fits month of Final Team Software Project, but with no respond from Glassdoor.
​
Glassdoor API request, used in the blog post, retrieves data about companies ratings. By reading [Glassdoor API Overview](https://www.glassdoor.com/developer/index.htm) and guidance in regards to parameters, we come to conclusion that by changing value of parameter ‘action’ from ‘action = employer’ to:
‘action = salaries’ we will fetch data about salaries, and
‘action = interviews’ we will fetch data about interview difficulty.
​
#### Job Postings Data
We weren’t able to find a value of parameter ‘action’ that would retrieve data about job postings, hence we decided to use web scraping for that quest. Tech topics from Meetup were used to retrieve corresponding job posts for Dublin. 

To retrieve all the interview difficulty ratings that correspond to retrieved job posts we used Glassdoor API.

Parameters values in API request were set up in the following way:
* parameter ‘action’ was set up to have value ‘interviews’, and
* parameter 'employer' was set up to match Company Names retrieved from Job Postings Data, and
* parameter ‘JobTitle’ was set up to match either Designation retrieved from Job Postings Data or Tech topics from Meetup.

### Getting Interview Difficulty for the tech topics in the jobTitle.csv

In [61]:
URLs2=[]
for i in range(jobTitle_df.shape[0]):
    parametars={'t.p':25738,'t.k':'iRCtcWJQamE','userip':'0.0.0.0','format':'json','v':1,'action':'interviews','jobTitle':jobTitle_df['jobTitle'][i]}
    r = requests.get("http://api.glassdoor.com/api/api.htm", params=parametars)
    URLs2.append(r.url)

In [78]:
hdr = {'User-Agent': 'Mozilla/5.0'}
interview_list_nested2 = []
for i in range(len(URLs2)):
    url = URLs2[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested2.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i]) 

In [107]:
for i in range(len(jobTitle_column)):
    interview_list_nested2[i]['JobTitle_TechTopic']=jobTitle_column[i]

In [108]:
interview_list_nested2[6]

,exactMatch,id,industry,industryId,industryName,interviewQuestionCount,interviewReviewCount,lastReviewDateTime,name,referrers,sectorId,sectorName,sentiment,tech_topic,JobTitle_TechTopic
0,False,6036,Internet,200063,Internet,1,1,,Amazon,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10013,Information Technology,"{'averageOverallDifficulty': 3, 'overallDiffic...",Amazon Web Services,Amazon Web Services


In [109]:
interviews_df2 = pd.DataFrame()
for i in range(len(interview_list_nested2)):
    interviews_df2=interviews_df2.append(interview_list_nested2[i])
interviews_df2.head()

,JobTitle_TechTopic,exactMatch,id,industry,industryId,industryName,interviewQuestionCount,interviewReviewCount,lastReviewDateTime,name,referrers,sectorId,sectorName,sentiment,tech_topic
0,Amazon Web Services,False,6036.0,Internet,200063.0,Internet,1.0,1.0,,Amazon,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10013.0,Information Technology,"{'averageOverallDifficulty': 3, 'overallDiffic...",Amazon Web Services
0,Android,False,1459414.0,Internet,200063.0,Internet,1.0,1.0,,Arekibo,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10013.0,Information Technology,"{'averageOverallDifficulty': 4, 'overallDiffic...",Android
1,Android,False,41925.0,IT Services,200064.0,IT Services,3.0,1.0,,Travelport,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10013.0,Information Technology,"{'averageOverallDifficulty': 2, 'overallDiffic...",Android
2,Android,False,872598.0,Internet,200063.0,Internet,1.0,1.0,,Cleverbug,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10013.0,Information Technology,"{'averageOverallDifficulty': 2, 'overallDiffic...",Android
3,Android,False,1140416.0,Enterprise Software & Network Solutions,200061.0,Enterprise Software & Network Solutions,1.0,1.0,,Digisoft.TV,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10013.0,Information Technology,"{'averageOverallDifficulty': 3, 'overallDiffic...",Android


In [110]:
split_sentiment = interviews_df2['sentiment'].apply(pd.Series) # split the sentiment column from split_sentiment
split_sentiment = split_sentiment.reset_index(drop=True) # reset index

interviews_df2['averageOverallDifficulty']=split_sentiment['averageOverallDifficulty']
interviews_df2['overallDifficultyCount']=split_sentiment['overallDifficultyCount']
interviews_df2['overallExperienceCount']=split_sentiment['overallExperienceCount']
interviews_df2['overallPosExperiencePercent']=split_sentiment['overallPosExperiencePercent']
interviews_df2['overallPosExperienceCount']=split_sentiment['overallPosExperienceCount']
interviews_df2['overallNeutExperiencePercent']=split_sentiment['overallNeutExperiencePercent']
interviews_df2['overallNeutExperienceCount']=split_sentiment['overallNeutExperienceCount']
interviews_df2['overallNegExperiencePercent']=split_sentiment['overallNegExperiencePercent']
interviews_df2['overallNegExperienceCount']=split_sentiment['overallNegExperienceCount']

In [111]:
interviews_df2=interviews_df2.drop(['sentiment','referrers'], axis=1)

In [114]:
interviews_df2=interviews_df2.drop(['tech_topic'], axis=1)

In [115]:
interviews_df2.to_csv('interviews2_tech_topics.csv', sep = ',', index = False) # create csv for interview_difficulty
interviews_df2

,JobTitle_TechTopic,exactMatch,id,industry,industryId,industryName,interviewQuestionCount,interviewReviewCount,lastReviewDateTime,name,...,sectorName,averageOverallDifficulty,overallDifficultyCount,overallExperienceCount,overallPosExperiencePercent,overallPosExperienceCount,overallNeutExperiencePercent,overallNeutExperienceCount,overallNegExperiencePercent,overallNegExperienceCount
0,Amazon Web Services,False,6036.0,Internet,200063.0,Internet,1.0,1.0,,Amazon,...,Information Technology,3.0,1.0,1.0,100.0,1.0,0.0,0.0,0.0,0.0
0,Android,False,1459414.0,Internet,200063.0,Internet,1.0,1.0,,Arekibo,...,Information Technology,3.0,1.0,1.0,100.0,1.0,0.0,0.0,0.0,0.0
1,Android,False,41925.0,IT Services,200064.0,IT Services,3.0,1.0,,Travelport,...,Information Technology,4.0,1.0,1.0,100.0,1.0,0.0,0.0,0.0,0.0
2,Android,False,872598.0,Internet,200063.0,Internet,1.0,1.0,,Cleverbug,...,Information Technology,2.0,1.0,1.0,0.0,0.0,100.0,1.0,0.0,0.0
3,Android,False,1140416.0,Enterprise Software & Network Solutions,200061.0,Enterprise Software & Network Solutions,1.0,1.0,,Digisoft.TV,...,Information Technology,2.0,1.0,1.0,0.0,0.0,0.0,0.0,100.0,1.0
0,Angular,False,229836.0,IT Services,200064.0,IT Services,1.0,1.0,,Verizon Telematics,...,Information Technology,3.0,1.0,1.0,100.0,1.0,0.0,0.0,0.0,0.0
0,Big Data,False,17494.0,Enterprise Software & Network Solutions,200061.0,Enterprise Software & Network Solutions,1.0,1.0,,Tenable,...,Information Technology,3.0,1.0,1.0,100.0,1.0,0.0,0.0,0.0,0.0
1,Big Data,False,232.0,Financial Analytics & Research,200051.0,Financial Analytics & Research,1.0,1.0,,Equifax,...,Finance,4.0,1.0,1.0,100.0,1.0,0.0,0.0,0.0,0.0
0,Blockchain,False,2763.0,Accounting,200001.0,Accounting,1.0,1.0,,Deloitte,...,Accounting & Legal,3.0,1.0,1.0,100.0,1.0,0.0,0.0,0.0,0.0
0,C#,False,295128.0,Enterprise Software & Network Solutions,200061.0,Enterprise Software & Network Solutions,1.0,1.0,,Arista Networks,...,Information Technology,3.0,1.0,1.0,100.0,1.0,0.0,0.0,0.0,0.0


### Getting Interviews for the jobTitles in Designation.csv

In [113]:
URLs3=[]
for i in range(designation_df.shape[0]):
    parametars={'t.p':25738,'t.k':'iRCtcWJQamE','userip':'0.0.0.0','format':'json','v':1,'action':'interviews','jobTitle':designation_df['Designation'][i]}
    r = requests.get("http://api.glassdoor.com/api/api.htm", params=parametars)
    URLs3.append(r.url)

In [137]:
jobTitle_column=[]
jobTitle_column_df=pd.DataFrame()

In [138]:
interview_list_nested3 = []
for i in range(300):
    url = URLs3[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 3 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested3.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_df['Designation'][i]}, ignore_index=True)

In [146]:
for i in range(300,600):
    url = URLs3[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested3.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_df['Designation'][i]}, ignore_index=True)

In [147]:
for i in range(600,1000):
    url = URLs3[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested3.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_df['Designation'][i]}, ignore_index=True)

In [148]:
for i in range(1000,1500):
    url = URLs3[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested3.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_df['Designation'][i]}, ignore_index=True)

In [149]:
for i in range(1500,2000):
    url = URLs3[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested3.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_df['Designation'][i]}, ignore_index=True)

In [150]:
for i in range(2000,2500):
    url = URLs3[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested3.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_df['Designation'][i]}, ignore_index=True)

In [151]:
for i in range(2500,3000):
    url = URLs3[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested3.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_df['Designation'][i]}, ignore_index=True)

In [152]:
for i in range(3000,3500):
    url = URLs3[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested3.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_df['Designation'][i]}, ignore_index=True)

In [154]:
for i in range(3500,3800):
    url = URLs3[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested3.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_df['Designation'][i]}, ignore_index=True)

In [155]:
for i in range(3800,len(URLs3)):
    url = URLs3[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested3.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_df['Designation'][i]}, ignore_index=True)

In [156]:
for i in range(len(URLs3)):
    interview_list_nested3[i]['JobTitle_TechTopic']=jobTitle_column_df['jobTitle'][i]

In [157]:
interviews_df3 = pd.DataFrame()
for i in range(len(interview_list_nested3)):
    interviews_df3=interviews_df3.append(interview_list_nested3[i])
interviews_df3.head()

,JobTitle_TechTopic,exactMatch,id,industry,industryId,industryName,interviewQuestionCount,interviewReviewCount,lastReviewDateTime,name,referrers,sectorId,sectorName,sentiment
0,ACCOUNT MANAGER,False,9079.0,Internet,200063.0,Internet,43.0,39.0,,Google,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10013.0,Information Technology,"{'averageOverallDifficulty': 3.3513513, 'overa..."
1,ACCOUNT MANAGER,False,40772.0,Internet,200063.0,Internet,45.0,34.0,,Facebook,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10013.0,Information Technology,"{'averageOverallDifficulty': 3.3939395, 'overa..."
2,ACCOUNT MANAGER,False,100569.0,Internet,200063.0,Internet,8.0,8.0,,Twitter,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10013.0,Information Technology,"{'averageOverallDifficulty': 3.5, 'overallDiff..."
3,ACCOUNT MANAGER,False,1651.0,Computer Hardware & Software,200060.0,Computer Hardware & Software,6.0,6.0,,Microsoft,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10013.0,Information Technology,"{'averageOverallDifficulty': 3.3333333, 'overa..."
4,ACCOUNT MANAGER,False,5944.0,Staffing & Outsourcing,200032.0,Staffing & Outsourcing,7.0,6.0,,TTEC,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10006.0,Business Services,"{'averageOverallDifficulty': 2.3333333, 'overa..."


In [158]:
split_sentiment = interviews_df3['sentiment'].apply(pd.Series) # split the sentiment column from split_sentiment
split_sentiment = split_sentiment.reset_index(drop=True) # reset index

interviews_df3['averageOverallDifficulty']=split_sentiment['averageOverallDifficulty']
interviews_df3['overallDifficultyCount']=split_sentiment['overallDifficultyCount']
interviews_df3['overallExperienceCount']=split_sentiment['overallExperienceCount']
interviews_df3['overallPosExperiencePercent']=split_sentiment['overallPosExperiencePercent']
interviews_df3['overallPosExperienceCount']=split_sentiment['overallPosExperienceCount']
interviews_df3['overallNeutExperiencePercent']=split_sentiment['overallNeutExperiencePercent']
interviews_df3['overallNeutExperienceCount']=split_sentiment['overallNeutExperienceCount']
interviews_df3['overallNegExperiencePercent']=split_sentiment['overallNegExperiencePercent']
interviews_df3['overallNegExperienceCount']=split_sentiment['overallNegExperienceCount']

In [159]:
interviews_df3=interviews_df3.drop(['sentiment','referrers'], axis=1)

In [160]:
interviews_df3.to_csv('interviews3_designation.csv', sep = ',', index = False) # create csv for interview_difficulty
interviews_df3

,JobTitle_TechTopic,exactMatch,id,industry,industryId,industryName,interviewQuestionCount,interviewReviewCount,lastReviewDateTime,name,...,sectorName,averageOverallDifficulty,overallDifficultyCount,overallExperienceCount,overallPosExperiencePercent,overallPosExperienceCount,overallNeutExperiencePercent,overallNeutExperienceCount,overallNegExperiencePercent,overallNegExperienceCount
0,ACCOUNT MANAGER,False,9079.0,Internet,200063.0,Internet,43.0,39.0,,Google,...,Information Technology,3.351351,37.0,35.0,82.85715,29.0,5.714286,2.0,11.428572,4.0
1,ACCOUNT MANAGER,False,40772.0,Internet,200063.0,Internet,45.0,34.0,,Facebook,...,Information Technology,3.393940,33.0,33.0,72.72727,24.0,9.090909,3.0,18.181818,6.0
2,ACCOUNT MANAGER,False,100569.0,Internet,200063.0,Internet,8.0,8.0,,Twitter,...,Information Technology,3.500000,8.0,7.0,28.57143,2.0,0.000000,0.0,71.428570,5.0
3,ACCOUNT MANAGER,False,1651.0,Computer Hardware & Software,200060.0,Computer Hardware & Software,6.0,6.0,,Microsoft,...,Information Technology,3.333333,6.0,6.0,50.00000,3.0,16.666668,1.0,33.333336,2.0
4,ACCOUNT MANAGER,False,5944.0,Staffing & Outsourcing,200032.0,Staffing & Outsourcing,7.0,6.0,,TTEC,...,Business Services,2.333333,6.0,6.0,66.66667,4.0,16.666668,1.0,16.666668,1.0
5,ACCOUNT MANAGER,False,9848.0,Internet,200063.0,Internet,5.0,5.0,,PayPal,...,Information Technology,3.200000,5.0,5.0,100.00000,5.0,0.000000,0.0,0.000000,0.0
6,ACCOUNT MANAGER,False,1327.0,IT Services,200064.0,IT Services,5.0,5.0,,Dell,...,Information Technology,3.000000,5.0,5.0,100.00000,5.0,0.000000,0.0,0.000000,0.0
7,ACCOUNT MANAGER,False,394648.0,Advertising & Marketing,200022.0,Advertising & Marketing,4.0,4.0,,Sojern,...,Business Services,2.666667,3.0,4.0,50.00000,2.0,0.000000,0.0,50.000000,2.0
8,ACCOUNT MANAGER,False,328783.0,Advertising & Marketing,200022.0,Advertising & Marketing,4.0,4.0,,AdRoll Group,...,Business Services,3.000000,4.0,4.0,25.00000,1.0,50.000000,2.0,25.000000,1.0
9,ACCOUNT MANAGER,False,100431.0,Internet,200063.0,Internet,7.0,4.0,,Glassdoor,...,Information Technology,3.250000,4.0,4.0,75.00000,3.0,0.000000,0.0,25.000000,1.0


### Merging two dataframes into one (designation with companies and designations)

In [161]:
interviews_designation_tech_topics_df=interviews_df3.append(interviews_df2)

In [162]:
interviews_designation_tech_topics_df.to_csv('interviews4_designation_tech_topics.csv', sep = ',', index = False) # create csv for interviews
interviews_designation_tech_topics_df

,JobTitle_TechTopic,exactMatch,id,industry,industryId,industryName,interviewQuestionCount,interviewReviewCount,lastReviewDateTime,name,...,sectorName,averageOverallDifficulty,overallDifficultyCount,overallExperienceCount,overallPosExperiencePercent,overallPosExperienceCount,overallNeutExperiencePercent,overallNeutExperienceCount,overallNegExperiencePercent,overallNegExperienceCount
0,ACCOUNT MANAGER,False,9079.0,Internet,200063.0,Internet,43.0,39.0,,Google,...,Information Technology,3.351351,37.0,35.0,82.85715,29.0,5.714286,2.0,11.428572,4.0
1,ACCOUNT MANAGER,False,40772.0,Internet,200063.0,Internet,45.0,34.0,,Facebook,...,Information Technology,3.393940,33.0,33.0,72.72727,24.0,9.090909,3.0,18.181818,6.0
2,ACCOUNT MANAGER,False,100569.0,Internet,200063.0,Internet,8.0,8.0,,Twitter,...,Information Technology,3.500000,8.0,7.0,28.57143,2.0,0.000000,0.0,71.428570,5.0
3,ACCOUNT MANAGER,False,1651.0,Computer Hardware & Software,200060.0,Computer Hardware & Software,6.0,6.0,,Microsoft,...,Information Technology,3.333333,6.0,6.0,50.00000,3.0,16.666668,1.0,33.333336,2.0
4,ACCOUNT MANAGER,False,5944.0,Staffing & Outsourcing,200032.0,Staffing & Outsourcing,7.0,6.0,,TTEC,...,Business Services,2.333333,6.0,6.0,66.66667,4.0,16.666668,1.0,16.666668,1.0
5,ACCOUNT MANAGER,False,9848.0,Internet,200063.0,Internet,5.0,5.0,,PayPal,...,Information Technology,3.200000,5.0,5.0,100.00000,5.0,0.000000,0.0,0.000000,0.0
6,ACCOUNT MANAGER,False,1327.0,IT Services,200064.0,IT Services,5.0,5.0,,Dell,...,Information Technology,3.000000,5.0,5.0,100.00000,5.0,0.000000,0.0,0.000000,0.0
7,ACCOUNT MANAGER,False,394648.0,Advertising & Marketing,200022.0,Advertising & Marketing,4.0,4.0,,Sojern,...,Business Services,2.666667,3.0,4.0,50.00000,2.0,0.000000,0.0,50.000000,2.0
8,ACCOUNT MANAGER,False,328783.0,Advertising & Marketing,200022.0,Advertising & Marketing,4.0,4.0,,AdRoll Group,...,Business Services,3.000000,4.0,4.0,25.00000,1.0,50.000000,2.0,25.000000,1.0
9,ACCOUNT MANAGER,False,100431.0,Internet,200063.0,Internet,7.0,4.0,,Glassdoor,...,Information Technology,3.250000,4.0,4.0,75.00000,3.0,0.000000,0.0,25.000000,1.0


### Getting Interviews for the companies in the Employers.csv and for the designation in the Designation.csv

In [169]:
URLs4=[]
for i in range(designation_company_df.shape[0]):
    parametars={'t.p':25738,'t.k':'iRCtcWJQamE','userip':'0.0.0.0','format':'json','v':1,'action':'interviews','employer':designation_company_df['Company'][i],'jobTitle':designation_company_df['Designation'][i]}
    r = requests.get("http://api.glassdoor.com/api/api.htm", params=parametars)
    #print(r.url)
    URLs4.append(r.url)

In [170]:
print (len(URLs4))
jobTitle_column_df=pd.DataFrame()

8490


In [184]:
URLs4[1752]

'http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=interviews&employer=Citi&jobTitle=Analyst'

In [172]:
interview_list_nested4 = []
for i in range(500):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [173]:
for i in range(500, 1000):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [174]:
for i in range(1000, 1500):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [178]:
for i in range(1500, 1750):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [181]:
for i in range(1750, 1752):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [ ]:
#'http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=interviews&employer=Citi&jobTitle=Analyst'
for i in range(1752, 1753): #KeyError: 'response'
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [183]:
for i in range(1753, 2000):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [187]:
for i in range(2000, 2500):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [188]:
for i in range(2500, 3000):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [189]:
for i in range(3000, 3500):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [190]:
for i in range(3500, 4000):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [191]:
for i in range(4000, 4500):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])):

In [193]:
for i in range(4500, 5000):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [194]:
for i in range(5000, 5500):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [195]:
for i in range(5500, 6000):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [196]:
for i in range(6000, 6500):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [199]:
for i in range(6500, 6750):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [201]:
for i in range(6750, 6800):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [203]:
for i in range(6800, 6850):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [204]:
for i in range(6850, 6875):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [206]:
for i in range(6875, 6880):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [ ]:
for i in range(6880, 6881):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [ ]:
for i in range(6881, 6900):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [ ]:
for i in range(6900, 6905):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [222]:
for i in range(6905, 7000):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [223]:
for i in range(7000, 7500):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [225]:
for i in range(7500, 8000):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [226]:
for i in range(8000, len(URLs4)):
    url = URLs4[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep( 1 )
    soup = BeautifulSoup(response, 'lxml')
    interview_data=json.loads(soup.p.get_text())
    interview_list_nested4.append(pd.DataFrame(interview_data['response']['employers']))
    jobTitle_column_df = jobTitle_column_df.append({'jobTitle': designation_company_df['Designation'][i]}, ignore_index=True)
    #jobTitle_column.append(pd.DataFrame(jobTitle_df)['jobTitle'][i])

In [227]:
for i in range(len(URLs4)):
    interview_list_nested4[i]['JobTitle_TechTopic']=jobTitle_column_df['jobTitle'][i]

In [228]:
interviews_df4 = pd.DataFrame()
for i in range(len(interview_list_nested4)):
    interviews_df4=interviews_df4.append(interview_list_nested4[i])
interviews_df4.head()

,JobTitle_TechTopic,exactMatch,id,industry,industryId,industryName,interviewQuestionCount,interviewReviewCount,lastReviewDateTime,name,referrers,sectorId,sectorName,sentiment
0,ACCOUNT MANAGER,True,357890.0,Construction,200036.0,Construction,0.0,0.0,,Nan,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10007.0,"Construction, Repair & Maintenance","{'averageOverallDifficulty': 0, 'overallDiffic..."
1,ACCOUNT MANAGER,False,1098257.0,,NaN,NaN,0.0,0.0,,Nan and Byron's,"{'overallAppliedOnlinePercent': 0, 'overallApp...",NaN,NaN,"{'averageOverallDifficulty': 0, 'overallDiffic..."
2,ACCOUNT MANAGER,False,519685.0,,NaN,NaN,0.0,0.0,,N. A. N. Woollen Mills,"{'overallAppliedOnlinePercent': 0, 'overallApp...",NaN,NaN,"{'averageOverallDifficulty': 0, 'overallDiffic..."
3,ACCOUNT MANAGER,False,355905.0,Advertising & Marketing,200022.0,Advertising & Marketing,0.0,0.0,,Don-Nan Pump and Supply,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10006.0,Business Services,"{'averageOverallDifficulty': 0, 'overallDiffic..."
4,ACCOUNT MANAGER,False,40837.0,Banks & Credit Unions,200048.0,Banks & Credit Unions,0.0,0.0,,Hua Nan Financial,"{'overallAppliedOnlinePercent': 0, 'overallApp...",10010.0,Finance,"{'averageOverallDifficulty': 0, 'overallDiffic..."


In [229]:
split_sentiment = interviews_df4['sentiment'].apply(pd.Series) # split the sentiment column from split_sentiment
split_sentiment = split_sentiment.reset_index(drop=True) # reset index

interviews_df4['averageOverallDifficulty']=split_sentiment['averageOverallDifficulty']
interviews_df4['overallDifficultyCount']=split_sentiment['overallDifficultyCount']
interviews_df4['overallExperienceCount']=split_sentiment['overallExperienceCount']
interviews_df4['overallPosExperiencePercent']=split_sentiment['overallPosExperiencePercent']
interviews_df4['overallPosExperienceCount']=split_sentiment['overallPosExperienceCount']
interviews_df4['overallNeutExperiencePercent']=split_sentiment['overallNeutExperiencePercent']
interviews_df4['overallNeutExperienceCount']=split_sentiment['overallNeutExperienceCount']
interviews_df4['overallNegExperiencePercent']=split_sentiment['overallNegExperiencePercent']
interviews_df4['overallNegExperienceCount']=split_sentiment['overallNegExperienceCount']

In [230]:
#Removing columns 'sentiment' and 'referrers'
interviews_df4=interviews_df4.drop(['sentiment','referrers'], axis=1)

In [231]:
interviews_df4.to_csv('interviews5_designation_companies.csv', sep = ',', index = False) # create csv for interview_difficulty
interviews_df4

,JobTitle_TechTopic,exactMatch,id,industry,industryId,industryName,interviewQuestionCount,interviewReviewCount,lastReviewDateTime,name,...,sectorName,averageOverallDifficulty,overallDifficultyCount,overallExperienceCount,overallPosExperiencePercent,overallPosExperienceCount,overallNeutExperiencePercent,overallNeutExperienceCount,overallNegExperiencePercent,overallNegExperienceCount
0,ACCOUNT MANAGER,True,357890.0,Construction,200036.0,Construction,0.0,0.0,,Nan,...,"Construction, Repair & Maintenance",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ACCOUNT MANAGER,False,1098257.0,,NaN,NaN,0.0,0.0,,Nan and Byron's,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ACCOUNT MANAGER,False,519685.0,,NaN,NaN,0.0,0.0,,N. A. N. Woollen Mills,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ACCOUNT MANAGER,False,355905.0,Advertising & Marketing,200022.0,Advertising & Marketing,0.0,0.0,,Don-Nan Pump and Supply,...,Business Services,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ACCOUNT MANAGER,False,40837.0,Banks & Credit Unions,200048.0,Banks & Credit Unions,0.0,0.0,,Hua Nan Financial,...,Finance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,ACCOUNT MANAGER,False,1909467.0,Education Training Services,200045.0,Education Training Services,0.0,0.0,,NAN Newark Tech World,...,Education,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,ACCOUNT MANAGER,False,294275.0,Transportation Equipment Manufacturing,200075.0,Transportation Equipment Manufacturing,0.0,0.0,,Nan Fang Distribution Group,...,Manufacturing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,ACCOUNT MANAGER,False,404481.0,Electrical & Electronic Manufacturing,200070.0,Electrical & Electronic Manufacturing,0.0,0.0,,Nan Ya PCB,...,Manufacturing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,ACCOUNT MANAGER,False,620481.0,"Cable, Internet & Telephone Providers",200120.0,"Cable, Internet & Telephone Providers",0.0,0.0,,NAN Star,...,Telecommunications,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,ACCOUNT MANAGER,False,8649.0,Chemical Manufacturing,200068.0,Chemical Manufacturing,0.0,0.0,,Nan Ya Plastics,...,Manufacturing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Merging all dataframes into one (companies+designation df with designation df and tech topics df)

In [232]:
interviews_all_df=interviews_df4.append(interviews_designation_tech_topics_df)

In [233]:
#Removing duplicate rows
interviews_all_df=interviews_all_df.drop_duplicates(subset=None, keep='first', inplace=False) 

In [234]:
interviews_all_df.to_csv('interviews_all.csv', sep = ',', index = False) # create csv for interviews
interviews_all_df

,JobTitle_TechTopic,exactMatch,id,industry,industryId,industryName,interviewQuestionCount,interviewReviewCount,lastReviewDateTime,name,...,sectorName,averageOverallDifficulty,overallDifficultyCount,overallExperienceCount,overallPosExperiencePercent,overallPosExperienceCount,overallNeutExperiencePercent,overallNeutExperienceCount,overallNegExperiencePercent,overallNegExperienceCount
0,ACCOUNT MANAGER,True,357890.0,Construction,200036.0,Construction,0.0,0.0,,Nan,...,"Construction, Repair & Maintenance",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ACCOUNT MANAGER,False,1098257.0,,NaN,NaN,0.0,0.0,,Nan and Byron's,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ACCOUNT MANAGER,False,519685.0,,NaN,NaN,0.0,0.0,,N. A. N. Woollen Mills,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ACCOUNT MANAGER,False,355905.0,Advertising & Marketing,200022.0,Advertising & Marketing,0.0,0.0,,Don-Nan Pump and Supply,...,Business Services,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ACCOUNT MANAGER,False,40837.0,Banks & Credit Unions,200048.0,Banks & Credit Unions,0.0,0.0,,Hua Nan Financial,...,Finance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,ACCOUNT MANAGER,False,1909467.0,Education Training Services,200045.0,Education Training Services,0.0,0.0,,NAN Newark Tech World,...,Education,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,ACCOUNT MANAGER,False,294275.0,Transportation Equipment Manufacturing,200075.0,Transportation Equipment Manufacturing,0.0,0.0,,Nan Fang Distribution Group,...,Manufacturing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,ACCOUNT MANAGER,False,404481.0,Electrical & Electronic Manufacturing,200070.0,Electrical & Electronic Manufacturing,0.0,0.0,,Nan Ya PCB,...,Manufacturing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,ACCOUNT MANAGER,False,620481.0,"Cable, Internet & Telephone Providers",200120.0,"Cable, Internet & Telephone Providers",0.0,0.0,,NAN Star,...,Telecommunications,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,ACCOUNT MANAGER,False,8649.0,Chemical Manufacturing,200068.0,Chemical Manufacturing,0.0,0.0,,Nan Ya Plastics,...,Manufacturing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Future Scope - retrieving all interview difficulty data

* Retrieve interviews from all pages: parameter - pn.
Example:
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&useragent=&format=json&v=1&action=interviews&jobTitle=%27data%20scientist%27&pn=2